# Adaptive

In [ ]:
import adaptive

adaptive.notebook_extension()

In [ ]:
def func(x, wait=True):
    """Function with a sharp peak on a smooth background"""
    import numpy as np
    from time import sleep
    from random import randint

    x = np.asarray(x)
    a = 0.001
    if wait:
        sleep(np.random.randint(1, 3))
    return x + a**2/(a**2 + (x)**2) #+ np.random.rand(1)

## Local Process Pool (default)

In [ ]:
# Initialize the learner
learner = adaptive.learner.Learner1D(func)
learner.add_point(-1, func(-1))
learner.add_point(1, func(1))
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.1)
adaptive.live_plot(runner)

## ipyparallel

In [ ]:
import ipyparallel

client = ipyparallel.Client()

# Initialize the learner
learner = adaptive.learner.Learner1D(func)
learner.add_point(-1, func(-1))
learner.add_point(1, func(1))

runner = adaptive.Runner(learner, client, goal=lambda l: l.loss() < 0.1)
adaptive.live_plot(runner)